# Bases de datos y SQL

En los próximos minutos vamos a conocer los aspectos básicos para poder empezar a trabajar con bases de datos en Python. No vamos a introducir el lenguaje SQL, ya que se asume que el lector posee un conocimiento básico sobre qué es SQL y cómo se utiliza.

En el siguiente PEP ([PEP 249 - The Python Database API Specification](https://www.python.org/dev/peps/pep-0249/)) encontramos las instrucciones básicas para conocer cómo se debe interactuar con las bases de datos a través de Python. Aunque existen diferencias notables entre cada uno de los sistemas de gestión de bases de datos, debería ser similar el uso de distintas bases de datos como SQLite, PostgreeSQL o MySQL, etc.

El primer concepto que debe quedarle claro al lector es que el primer paso a realizar es importar un driver; esto es, el API para la base de datos específica que se desa utilizar. Las más utilizadas son: 
* [psycopg2](http://initd.org/psycopg/)
* [MySQL Connector/Python](https://dev.mysql.com/doc/connector-python/en/)
* [sqlite3](https://docs.python.org/2/library/sqlite3.html)

En este tutorial abordaremos el uso de Python con SQLite y SQLAlchemy.



> **_NOTA:_**  **Python Enhancement Proposal (PEP)**.
El desarrollo del lenguaje de programación Python se lleva a cabo a través del proceso *Python Enhancement Proposal (PEP)*, mediante el cual se proponen las nuevas características a la comunidad Python para poder llegar a un consenso. Los PEP destacados son revisados y comentados por la comunidad de Python y el consejo directivo. Como ejemplos, tenemos el *Python coding style* ([PEP 8](https://www.python.org/dev/peps/pep-0008/)) o el *Zen of Python* ([PEP 20](https://www.python.org/dev/peps/pep-0020/)).



### SQLite

SQLite es una librería desarrollada en C que proporciona una base de datos ligera basada en disque que no requiere un procesor servidor separado y permite acceder a una base de datos utilizando una variante del lenguaje SQL. Muchas aplicaciones pueden utilizar SQLite para almacenamiento interno. Las bases de datos SQLite son la columna vertebral de muchas aplicaciones, aunque siempre deben mantenerse pequeñas. Ademas, es posible hacer uso de SQLite para un prototipo y posteriormente portar el código a un sistema de gestión de base de datos más potente como PostgreSQL o Oracle. 


In [2]:
import sqlite3

#### Conexión a una base de datos

El primer paso que hay que hacer es realizar una conexión a la base de datos. Usualmente, será necesario conocer la siguiente información para realizar la conexión:
* hostname
* port
* username
* password
* database name

SQLite es una base de datos embebida que se encuentra almacenada en un fichero en disco, y debe ser operada por un único programa. Por tanto, para crear una conexión a una base de datos SQLite, únicamente es necesario apuntar a un fichero en disco.

In [3]:
DBPATH = 'people.db'
conn = sqlite3.connect(DBPATH)

En este punto, el lector observará que un fichero con nombre *people.db* ha sido creado en el directorio de trabajo actual.

El método connect devuelte un objeto de tipo conexión que ha sido nombrado conn. A través del objeto conn se podrá manipular la conexión con la base de datos incluyendo los siguientes métodos:
* conn.commit() - commit any changes back to the database
* conn.close() - close our connection to the database and tidy up

Sin embargo, para ejecutar una sentencia SQL contra la base de datos para insertar --INSERT-- o seleccionar registros --SELECT--, es necesario crear un cursor:

In [4]:
cursor = conn.cursor()

Un cursor es esencialmente un puntero a la base de datos; un cursor posee los siguientes métodos:
* cursor.execute() - executes a SQL string against the database
* cursor.fetchone() - fetch a single row back from the executed query
* cursor.fetchall() - fetch all results back from the executed query

Juntos, cursores y conexiones conforman la manera básica de interactuar con una base de datos SQLite.


#### Descripción de la base de datos

El primer paso que hay que llevar a cabo es describir el tipo de datos que vamos a manejar en la base de datos mediante la creación de un *schema*. Para este taller, crearemos una aplicación simple de contactos; el esquema será el siguiente:


<img src="./database-contacts.png" width="400">


En la imagen se observan dos tablas, contactos --nombre, email y compañia-- y compañías/organizaciones/empresas --nombre--. Para crear la tabla de empresas ejecutaríamos la sentencia SQL de la siguiente manera:

In [5]:
sql = (
    "CREATE TABLE IF NOT EXISTS companies ("
    "    id INTEGER PRIMARY KEY AUTOINCREMENT,"
    "    name TEXT NOT NULL"
    ")"
)

cursor.execute(sql)

A continuación, se deberá escribir y ejecutar la sentencia SQL para crear la tabla de contactos.

In [6]:
# Create the contacts table

#### Inserción de registros

Una vez creada la base de datos con el esquema deseado, el siguiente paso es insertar registros en ella; Vamos a añadir la Georgetown University a la tabla de compañías.

In [7]:
sql = "INSERT INTO companies (name) VALUES (?)"
cursor.execute(sql, ("Georgetown University",))
conn.commit()

En las líneas de código anteriores, se ha creado esencialmente una plantilla SQL para insertar los nombres de las compañías en la tabla correspondiente. Como habrá podido observar el lector, no se ha asignado un id, ya que se generará de manera automática mediante la propiedad AUTOINCREMENT de ese campo de la base de datos.

El carácter ? es un parámetro de la sentencia SQL, y puede ser utilizado como un parámetro de sustitución para cualquier entrada que realice el usuario. Los valores para el parámetro son pasados como el segundo argumento como una tupla. En general, no deberían utilizarse métodos de formateo de cadenas string tales como:

> sql = "INSERT INTO companies (name) VALUES ({})".format("Georgetown University")

Esta forma de proceder es potencialmente no segura. Por otro lado, los parámetros ? realizan una gran cantidad de trabajo por detrás para asegurar que el comportamiento es correcto y seguro.

Vamos a continuar insertando otro registro usando la misma instrucción SQL.

In [8]:
cursor.execute(sql, ("US Department of Commerce",))
conn.commit()

El último punto que vamos a tratar es la llamada al método commit. Nada es escrito en la base de datos hasta que es llamado el método commit. Esta forma de proceder proporciona una habilidad interesante a la hora de realizar transacciones; una serie de consultas SQL se completarán de forma correcta y conjunta una vez realizamos el commit. Sin embargo, si algo no funciona de la manera esperada durante la ejecución, no se realizará la llamada a commit y se realizará, de facto, un *rollback*.


#### Selección de registros

Antes de continuar con la insercción de contactos, debemos conocer el identificador de la compañía del contacto que se desea insertar en la base de datos. Sin embargo, como se ha insertado mediante el uso de la característica de autoincremento, no es conocido el valor de los identificadores de la tabla que contiene los datos de las compañías. Para leer los datos, tendrán que ser buscados de la siguiente manera:

In [10]:
cursor.execute("SELECT id FROM companies WHERE name=?", ("Georgetown University",))
print(cursor.fetchone())

(1,)


Para obtener el registro deseado, se hace uso de la selección de registros mediante la cláusula SELECT con el parámetro "Georgetown University" --tupla--.

El método fetchone obtiene el primer registro que encuentra. Hay que destacar que el nombre de las compañías no han sido definidos como únicos, por tanto, podrían haber varios registros con nombre "Georgetown University". Si se desea obtener todos los registros se puede realizar mediante el método fetchall.

Para insertar un contacto que trabaja en la Georgetown University se deberá ejecutar una sentencia similar a la siguiente:

In [12]:
sql = "INSERT INTO contacts (name, email, company_id) VALUES (?,?,?)" 
cursor.execute(sql, ("Benjamin Bengfort", "bb830@georgetown.edu", 1))
conn.commit()

OperationalError: no such table: contacts

A partir de este momento, el lector debe ser capaz de insertar algunos contactos y compañías.

In [11]:
# Insert some contacts and companies using the methods described above.

##### Workshop

A partir de este momento, el lector deberá ser capaz de realizar una pequeña aplicación en Python que realice los siguientes requerimientos:
* Insertar contactos en la base de datos añadiendo el nombre, email y el nómbre de la compañía
* Mostrar la lista de compañías y el número de contactos asociados a ella

Utilizar el siguiente fragmento de código incompleto como plantilla para la realización del *workshop*:

In [14]:
import os
import sqlite3

def create_tables(conn):
    """
    Write your CREATE TABLE statements in this function and execute
    them with the passed in connection. 
    """
    # TODO: fill in. 
    pass


def connect(path="people.db", syncdb=False):
    """
    Connects to the database and ensures there are tables.
    """
    
    # Check if the SQLite file exists, if not create it.
    if not os.path.exists(path):
        syncdb=True

    # Connect to the sqlite database
    conn = sqlite3.connect(path)
    if syncdb:
        create_tables(conn)
    
    return conn


def insert(name, email, company, conn=None):
    if not conn: conn = connect()

    # Attempt to select company by name first. 
    
    # If not exists, insert and select new id.
    
    # Insert contact

    
if __name__ == "__main__":
    name    = raw_input("Enter name: ")
    email   = raw_input("Enter email: ")
    company = raw_input("Enter company: ")
    
    conn = connect()
    insert(name, email, company, conn)

    # Change below to count contacts per company! 
    contacts = conn.execute("SELECT count(id) FROM contacts").fetchone()
    print "There are now {} contacts".format(*contacts)

    conn.close()

SyntaxError: invalid syntax (<ipython-input-14-757cb19de80d>, line 50)


**Ejemplo:**

A continuación vamos a ver un ejemplo guiado en el que veremos el funcionamiento básico y el flujo de SQLite, todo ello mediante un código fuente _"pythonic"_. Trabajaremos con una base de datos con una única tabla que contendrá la información básica de jugadores de baloncesto de la NBA, de la siguiente manera:

<img src="./database-players.png" width="150">



In [6]:
# -*- coding: utf-8 -*-
import sqlite3
from player import Player

# Connect
conn = sqlite3.connect(':memory:')

# Create a cursor
c = conn.cursor()

# Create table
c.execute("""CREATE TABLE players (
             id integer PRIMARY KEY AUTOINCREMENT,
             first text,
             last text,
             height real,
             weight real,
             age integer
             )""")


def insert_player(player):
    with conn:
        c.execute("INSERT INTO players (first, last, height, weight, age) VALUES (:first, :last, :height, :weight, :age)",
                  {'first': player.first,
                  'last': player.last,
                  'height': player.height,
                  'weight': player.weight,
                  'age': player.age})


def get_players():
    c.execute("SELECT * FROM players")
    return c.fetchall()


def update_weight(player, weight):
    with conn:
        c.execute("""UPDATE players SET weight = :weight
                    WHERE first = :first AND last = :last""",
                  {'first': player.first, 'last': player.last, 'weight': player.weight})


def remove_player(player):
    with conn:
        c.execute("DELETE from players WHERE first = :first AND last = :last",
                  {'first': player.first, 'last': player.last})
        

# Players
player_1 = Player('Giannis', 'Antetokounmpo', 2.11, 109.8, 24)
player_2 = Player('Luka', 'Doncic', 1.96, 98.9, 20)
player_3 = Player('James', 'Harden', 1.96, 99.8, 29)

# Insert players into DB
insert_player(player_1)
insert_player(player_2)
insert_player(player_3)

# Get players
players = get_players()
print(players)

# Update info of some players
update_weight(player_2, 100.5)

# Remove player
remove_player(player_3)

# Get players
players = get_players()
print(players)

conn.close()

[(1, 'Giannis', 'Antetokounmpo', 2.11, 109.8, 24), (2, 'Luka', 'Doncic', 1.96, 98.9, 20), (3, 'James', 'Harden', 1.96, 99.8, 29)]
[(1, 'Giannis', 'Antetokounmpo', 2.11, 109.8, 24), (2, 'Luka', 'Doncic', 1.96, 98.9, 20)]


### SQLAlchemy

Referencias:
- [1] A brief tutorial on SQL with Python (using SQLite). https://github.com/georgetown-analytics/sql-tutorial
- [2] Python SQLite Tutorial: Complete Overview. https://www.youtube.com/watch?v=pd-0G0MigUA
- [3] Python SQLite (sqlite3). https://docs.python.org/2/library/sqlite3.html
